## Generating and analyzing images through the API

In [ ]:
%pip install ipython

In [8]:
import openai
from openai import  AssistantEventHandler, OpenAI
from typing_extensions import override
import os
import dotenv
from IPython.display import display, HTML

dotenv.load_dotenv(".env", override=True) 
openai.api_key = os.getenv("OPENAI_API_KEY")

## Creating an image based on a recipe

We will ask La Chef to provide a recipe and then have DALL-E 3 generate an image based on this recipe.

In [ ]:
### Create the assistant
client = OpenAI()
 
assistant = client.beta.assistants.create(
  name="La Chef",
  instructions="You are an expert cook. You have access to recipes of Bejo Zaden.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

# Create a vector store caled "Bejo recipes"
vector_store = client.beta.vector_stores.create(name="Bejo recipes")
 
# Ready the files for upload to OpenAI
file_paths = ["Recipes_Bejo.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# Create a thread
thread = client.beta.threads.create()



### The event handler

In [5]:
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        self.message = ""
        print(f"\nassistant running ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        self.message += message_content.value + "\n"
        self.message += "\n".join(citations)

### The conversation

In [ ]:
event_handler = EventHandler()
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please provide a recipe which includes some garlic and perhaps peppers.",
    event_handler=event_handler,
) as stream:
    stream.until_done()

print(event_handler.message)

### Generate the image with DALL-E 3

In [ ]:
response = client.images.generate(
  model="dall-e-3",
  prompt="Please provide a photo-realistic plate of food, given the following description and ingredients: `{event_handler.message}`",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
display(HTML(f'<img src="{image_url}" alt="Image" />'))

### Image of a stressed tomato plant

In [ ]:
response = client.images.generate(
  model="dall-e-3",
  prompt="Please provide a cartoon-like image of an unhappy tomato plant due to drought. The tomato is growing in between other, more happy, green tomato plants.",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
display(HTML(f'<img src="{image_url}" alt="Image" />'))

In [ ]:
response = client.images.generate(
  model="dall-e-3",
  prompt="Please provide a cartoon-like image of tomato plant with clear signs of drought.",
  size="1024x1024",
  quality="standard",
  n=1,
)

image_url = response.data[0].url
display(HTML(f'<img src="{image_url}" alt="Image" />'))

## Image analyzes

You can ask GTP-4 and GTP-4o to provide a description of an image. Generally speaking this is called image-to-text, but OpenAI calls is Vision. 

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "What’s in this image?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://visuals.rijkzwaan.com/m/15a977f5cedde13b/original/XX-Hero-L-VI24-Solutions-Industry.webp",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])